Rosette on Jupyter
====

Finding an integer whose absolute value is X with Racket

In [1]:
#lang iracket/lang #:require rosette/safe 

(define (abs x)
    (if (< x 0) (- x) x))

In [2]:
; define a symbolic variable y

(define-symbolic y integer?)

In [3]:
; Solve a constraint saying |y| = 5.
(solve
  (assert (= (abs y) 5)))

(model
 [y -5])